In [1]:
from torchkf import * 
import numpy as np
import scipy as sp
import sympy

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
""" format: value[, cov[, constaint]]
If constraint == 'positive', then the cov must be in log-space
""" 
parameters = dotdict(
    skeletal=dotdict(
        weight=dotdict(
            body=(70, np.exp(0), 'positive'), 
            relative=dotdict(
                forearm=(0.00762, np.exp(-4), 'positive'),
                upper_arm=(0.01373, np.exp(-4), 'positive'), 
                hand=(0.00361, np.exp(-4), 'positive'),
            ), 
        ), 
        size=dotdict(
            body=(1.75, np.exp(0), 'positive'), 
            relative=dotdict(
                forearm=(0.16480, np.exp(-4), 'positive'),
                upper_arm=(0.19590, np.exp(-4), 'positive'), 
                hand=(0.11465, np.exp(-4), 'positive'),
                humerus_factor=(0.8, np.exp(-4), 'positive'),
            ), 
        ),
        constraints=dotdict(
            elbow=dotdict(
                max_flexion=np.pi / 6,
                max_extension=9 * np.pi / 10.,
            )
        ),
        g=9.81,
    ),
    muscles=dotdict(
        triceps=dotdict(
            insertion=(0.02, np.exp(-8), 'positive'),
            tendon_length=(0.12, np.exp(-2), 'positive'),
            hill_muscle=dotdict(
                Fmax = (2397.12, np.exp(2), 'positive'), # N
                Lm0  = (0.15, np.exp(-2), 'positive'), # m
                phi0 = (0.2094, np.exp(-2)), # rad
                W    = (0.56, np.exp(-2), 'positive'),
                vmax = (1.3, np.exp(-2), 'positive'), # m/s
                A    = 0.25, 
                gmax = 1.5, 
                Lslack = (0.18, np.exp(0), 'positive'), # m
                Kmee1 = 10, 
                Kmee2 = 10_000,
                Tact  = 0.1,
            ),
        ), 
        biceps=dotdict(
            insertion=(0.02, np.exp(-8), 'positive'),
            tendon_length=(0.12, np.exp(-2), 'positive'),
            hill_muscle=dotdict(
                Fmax = (2874.67, np.exp(2), 'positive'), # N
                Lm0  = (0.13, np.exp(-2), 'positive'), # m
                phi0 = (0, np.exp(-2)), # rad
                W    = (0.56, np.exp(-2), 'positive'),
                vmax = (1.3, np.exp(-2), 'positive'), # m/s
                A    = 0.25, 
                gmax = 1.5, 
                Lslack = (0.21, np.exp(0), 'positive'), # m
                Kmee1 = 10, 
                Kmee2 = 10_000,
                Tact  = 0.1,
            ), 
        ),
    ), 
    afferents=dotdict(
        Ia=dotdict(
            kv=2.5,
            lmf=0.2,
            kdl=0.2,
            knl=0.5, 
            c1= 0.01,
        ), 
        Ib=dotdict(
            kf=1,
            fF=0.1,
        ),
    ),
)

In [3]:
def process_parameters(parameters): 
    proc_params = type(parameters)()
    for key, val in parameters.items(): 
        if isinstance(val, dict): 
            _proc_params = process_parameters(val)
            proc_params[key] = _proc_params
        else: 
            try: n = len(val)
            except TypeError: n = 0
            if n > 3: raise AttributeError(f'Parameter {key} has too many values (expected length 3)')
            param = [0, 0, None]
            
            # parameter value (prior expectation)
            if n == 0: param[0] = val
            else: param[0] = val[0]
                
            # parameter covariance
            if n > 1: param[1] = val[1]
                
            # parameter constraint
            if n == 3: param[2] = val[2]
            
            proc_params[key] = tuple(param)
    return proc_params
            
def flatten_parameters(parameters, namespace=''):
    flat_params = type(parameters)()
    for key, val in parameters.items(): 
        nkey = f'{namespace}.{key}' if len(namespace) > 0 else key
        if isinstance(val, dict): 
            _flat_params = flatten_parameters(val, namespace=nkey)
            flat_params.update(_flat_params)
        else: 
            flat_params[nkey] = val
    return flat_params            

In [4]:
import pandas as pd
df = pd.DataFrame(flatten_parameters(process_parameters(parameters))).transpose()
df = df.set_axis(['expectation', 'covariance', 'constraint'], axis=1)
df['constraint'] = df['constraint'].apply(lambda x: x if isinstance(x, str) else 'none')
df['covariance'] = df['covariance'].apply(lambda x: f'exp({int(np.log(x))})' if x > 0 else 0)
df

,expectation,covariance,constraint
skeletal.weight.body,70,exp(0),positive
skeletal.weight.relative.forearm,0.00762,exp(-4),positive
skeletal.weight.relative.upper_arm,0.01373,exp(-4),positive
skeletal.weight.relative.hand,0.00361,exp(-4),positive
skeletal.size.body,1.75,exp(0),positive
skeletal.size.relative.forearm,0.1648,exp(-4),positive
skeletal.size.relative.upper_arm,0.1959,exp(-4),positive
skeletal.size.relative.hand,0.11465,exp(-4),positive
skeletal.size.relative.humerus_factor,0.8,exp(-4),positive
skeletal.constraints.elbow.max_flexion,0.523599,0,none


In [5]:
from abc import ABC, abstractmethod

class NamedParamModel(ABC): 
    def __init__(self, 
                 state_names, input_names, param_names, 
                 state_map=None, input_map=None, param_map=None,
                 state_keys=None, input_keys=None, param_keys=None):
        
        self.state_names = state_names
        self.input_names = input_names
        self.param_names = param_names
        
        if state_map is None: 
            self.state_map = {k:k for k in self.state_names}
        else: 
            self.state_map = {
                k:state_map[k] if k in state_map.keys() else k 
                for k in state_names}
        
        if input_map is None: 
            self.input_map = {k:k for k in self.input_names}
        else: 
            self.input_map = {
                k:input_map[k] if k in input_map.keys() else k 
                for k in input_names}
            
        if param_map is None: 
            self.param_map = {k:k for k in self.param_names}
        else: 
            self.param_map = {
                k:param_map[k] if k in param_map.keys() else k 
                for k in param_names}
        
        if None not in (state_keys, input_keys, param_keys): 
            self.compute_indices(state_keys, input_keys, param_keys)
        
    def compute_indices(self, state_keys, input_keys, param_keys):
        self.state_keys = state_keys
        self.input_keys = input_keys
        self.param_keys = param_keys
        
        self.ix = [state_keys.index(self.state_map[k]) for k in self.state_names]
        self.iv = [input_keys.index(self.input_map[k]) for k in self.input_names]
        self.ip = [param_keys.index(self.param_map[k]) for k in self.param_names]
        
    @staticmethod
    def unvec_args(func): 
        from functools import wraps
        @wraps(func)
        def _wrap(self, x, v, p, **kwargs):
            try: 
                _x = x[:, 0][self.ix]
                _v = v[:, 0][self.iv]
                _p = p[:, 0][self.ip]
            except TypeError: 
                _x, _v, _p = x, v, p
            
            return func(self, *_x, *_v, *_p, **kwargs)
        return _wrap

In [6]:
def sheavyside(x, k=64): 
    return 1./(1 + sympy.exp(-2 * k * x))

def sclip(x, l, u, k=1): 
    return sheavyside((x-l)/(u-l), k) * (u - l) + l

In [16]:
class HillMuscle(NamedParamModel):  
    state_names = 'L', 'vel'
    input_names = 'aMN', 'gMN' 
    param_names = 'Kmee1', 'Kmee2', 'Lslack', 'Fmax', 'Lm0', 'W', 'vmax', 'A', 'gmax', 'phi0', 'kv', 'kdl', 'knl', 'c1', 'kf'
        
    def __init__(self, *args, **kwargs): 
        super().__init__(HillMuscle.state_names, HillMuscle.input_names, HillMuscle.param_names, *args, **kwargs)
    
    @staticmethod
    def force_velocity(vel, vmax, A, gmax): 
        cd    = vmax * A * (gmax - 1.) / (A + 1.)
        expr1 = (vmax + vel) / (vmax - vel/A)
        expr2 = (gmax * vel + cd) / (vel + cd)
#         h     = sheavyside(vel)
        
        return sympy.Piecewise((expr1, (vel <= 0)), (expr2, True))
#         return expr1
    
    @staticmethod
    def force_length(L, Lm0, W): 
        return sympy.exp(- (L - Lm0)**2/(W * Lm0)**2)
    
    @staticmethod
    def Fmee(L, Kmee1, Kmee2, Lslack):
        expr1 = Kmee1 * (L - Lslack)
        expr2 = Kmee2 * (L - Lslack)**2
#         h     = sheavyside(L - Lslack)
#         return expr1 + h * expr2
        return expr1 + sympy.Piecewise((0, (L < Lslack)), (expr2, True))
    
    @staticmethod
    def Fmce(aMN, Fmax, L, vel, Lm0, W, vmax, A, gmax): 
        fl = HillMuscle.force_length(L, Lm0, W)
        fv = HillMuscle.force_velocity(vel, vmax, A, gmax)
        
        return aMN * Fmax * fl * fv
    
    @staticmethod
    def Frte(L, vel, aMN, Kmee1, Kmee2, Lslack, Fmax, Lm0, W, vmax, A, gmax, phi0):     
        Fmee = HillMuscle.Fmee(L, Kmee1, Kmee2, Lslack)
        Fmce = HillMuscle.Fmce(aMN, Fmax, L, vel, Lm0, W, vmax, A, gmax)
        
        return (Fmee + Fmce) * sympy.cos(Lm0 * sympy.sin(phi0) / L)

    @staticmethod
    def rateIa(L, vel, gMN, Lm0, vmax, kv, kdl, knl, c1):
        dnorm = (L - 0.2 * Lm0) / Lm0
        vnorm = vel / vmax
        Ia    = kv * vnorm + kdl * dnorm + knl * gMN + c1 
        Ia    = sheavyside(Ia, k=1)
        
#         Ia    = sclip(Ia, 0, 1)
#         Ia    = sympy.Piecewise((0, Ia < 0),  (Ia, (Ia >= 0) & (Ia <= 1)), (1, True))
        return Ia
    
    @staticmethod
    def rateIb(Frte, Fmax, kf): 
        Fnorm = (Frte - 0.1 * Fmax) / Fmax
        Ib    = kf * Fnorm
        Ib    = sheavyside(Ib, k=1)
#         Ib    = sclip(Ib, 0, 1)
        
#         Ib    = sympy.Piecewise((0, Ib < 0), (Ib, (Ib >=0) & (Ib <=1)), (1, True))
        return Ib
    
    @NamedParamModel.unvec_args
    def compute_output(self, L, vel, aMN, gMN, Kmee1, Kmee2, Lslack, Fmax, Lm0, W, 
                       vmax, A, gmax, phi0, kv, kdl, knl, c1, kf, return_force=True, return_rates=True):      
        y = []
        
        Frte   = HillMuscle.Frte(L, vel, aMN, Kmee1, Kmee2, Lslack, Fmax, W, vmax, A, gmax, Lm0, phi0)
        if return_force: 
            y.append(Frte)
            
        if return_rates: 
            rateIa = HillMuscle.rateIa(L, vel, gMN, Lm0, vmax, kv, kdl, knl, c1)
            rateIb = HillMuscle.rateIb(Frte, Fmax, kf)       
            
            y.extend([rateIa, rateIb])
        if len(y) == 1:
            y, = y
        
        return y

In [17]:
flatten_parameters(process_parameters(parameters))

{'skeletal.weight.body': (70, 1.0, 'positive'),
 'skeletal.weight.relative.forearm': (0.00762,
  0.01831563888873418,
  'positive'),
 'skeletal.weight.relative.upper_arm': (0.01373,
  0.01831563888873418,
  'positive'),
 'skeletal.weight.relative.hand': (0.00361, 0.01831563888873418, 'positive'),
 'skeletal.size.body': (1.75, 1.0, 'positive'),
 'skeletal.size.relative.forearm': (0.1648, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.upper_arm': (0.1959, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.hand': (0.11465, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.humerus_factor': (0.8,
  0.01831563888873418,
  'positive'),
 'skeletal.constraints.elbow.max_flexion': (0.5235987755982988, 0, None),
 'skeletal.constraints.elbow.max_extension': (2.827433388230814, 0, None),
 'skeletal.g': (9.81, 0, None),
 'muscles.triceps.insertion': (0.02, 0.00033546262790251185, 'positive'),
 'muscles.triceps.tendon_length': (0.12, 0.1353352832366127, 'positive'),
 'mus

In [18]:
state_keys = 'L', 'vel'
input_keys = 'aMN', 'gMN'
params     = flatten_parameters(process_parameters(parameters))
param_keys = [*params.keys()]
param_map  = {k: f'muscles.triceps.hill_muscle.{k}' 
              for k in HillMuscle.param_names if f'muscles.triceps.hill_muscle.{k}' in param_keys}
param_map.update(
    {k: f'afferents.Ia.{k}' for k in HillMuscle.param_names if f'afferents.Ia.{k}' in param_keys})
param_map.update(
    {k: f'afferents.Ib.{k}' for k in HillMuscle.param_names if f'afferents.Ib.{k}' in param_keys})

triceps_muscle = HillMuscle(param_map=param_map ,state_keys=state_keys, input_keys=input_keys, param_keys=param_keys)

In [19]:
import pandas as pd
df = pd.DataFrame(flatten_parameters(process_parameters(parameters))).transpose()
df = df.set_axis(['expectation', 'covariance', 'constraint'], axis=1)
df['constraint'] = df['constraint'].apply(lambda x: x if isinstance(x, str) else 'none')
df['covariance'] = df['covariance'].apply(lambda x: f'exp({int(np.log(x))})' if x > 0 else 0)
df

,expectation,covariance,constraint
skeletal.weight.body,70,exp(0),positive
skeletal.weight.relative.forearm,0.00762,exp(-4),positive
skeletal.weight.relative.upper_arm,0.01373,exp(-4),positive
skeletal.weight.relative.hand,0.00361,exp(-4),positive
skeletal.size.body,1.75,exp(0),positive
skeletal.size.relative.forearm,0.1648,exp(-4),positive
skeletal.size.relative.upper_arm,0.1959,exp(-4),positive
skeletal.size.relative.hand,0.11465,exp(-4),positive
skeletal.size.relative.humerus_factor,0.8,exp(-4),positive
skeletal.constraints.elbow.max_flexion,0.523599,0,none


In [20]:
from collections import OrderedDict

In [115]:
class ElbowModel(NamedParamModel):
    state_names = 'q', 'dq'
    input_names = 'q2', 'aMNb', 'aMNt', 'gMNb', 'gMNt'
    output_names = 'q', 'aMNb', 'aMNt', 'gMNb', 'gMNt', 'rateIa_b', 'rateIb_b', 'rateIa_t', 'rateIb_t'
    output_keys = output_names
    param_map   = OrderedDict({
        'uarm_l_rel':    'skeletal.size.relative.upper_arm', 
        'farm_l_rel':    'skeletal.size.relative.forearm', 
        'hand_l_rel':    'skeletal.size.relative.hand', 
        'uarm_w_rel':    'skeletal.weight.relative.upper_arm',
        'farm_w_rel':    'skeletal.weight.relative.forearm', 
        'hand_w_rel':    'skeletal.weight.relative.hand', 
        'body_w':        'skeletal.weight.body', 
        'body_h':        'skeletal.size.body',
        'humerus_factor':'skeletal.size.relative.humerus_factor', 
        'lrte_b':        'muscles.biceps.tendon_length', 
        'lrte_t':        'muscles.triceps.tendon_length', 
        'g':             'skeletal.g', 
        'db':            'muscles.biceps.insertion', 
        'dt':            'muscles.triceps.insertion', 
    })
    
    def __init__(self, param_keys): 
        pmap = OrderedDict(ElbowModel.param_map)
        
        param_maps = dict()
        for muscle in ('biceps', 'triceps'): 
            param_maps[muscle] = {k: f'muscles.{muscle}.hill_muscle.{k}' 
                  for k in HillMuscle.param_names if f'muscles.{muscle}.hill_muscle.{k}' in param_keys}
            param_maps[muscle].update(
                {k: f'afferents.Ia.{k}' for k in HillMuscle.param_names if f'afferents.Ia.{k}' in param_keys})
            param_maps[muscle].update(
                {k: f'afferents.Ib.{k}' for k in HillMuscle.param_names if f'afferents.Ib.{k}' in param_keys})
        
        self.biceps_model  = HillMuscle(param_map=param_maps['biceps'], state_keys=HillMuscle.state_names, input_keys=HillMuscle.input_names, param_keys=param_keys)
        self.triceps_model = HillMuscle(param_map=param_maps['triceps'], state_keys=HillMuscle.state_names, input_keys=HillMuscle.input_names, param_keys=param_keys)
        
        pmap.update({f'biceps.{k}': v for k, v in self.biceps_model.param_map.items()})
        pmap.update({f'triceps.{k}': v for k, v in self.triceps_model.param_map.items()})
        
        pnames = (*pmap.keys(),)
        
        super().__init__(ElbowModel.state_names, ElbowModel.input_names, pnames, param_map=pmap, 
                        state_keys=ElbowModel.state_names, input_keys=ElbowModel.input_names, param_keys=param_keys)
        
        self.n  = len(self.state_names)
        self.m  = len(self.input_names)
        self.nP = len(self.param_names)
    
    @staticmethod
    def body_mensurations(uarm_l_rel, farm_l_rel, hand_l_rel, uarm_w_rel, farm_w_rel, hand_w_rel, 
                          body_w, body_h, humerus_factor):
        uarm_l, farm_l, hand_l = (rel * body_h for rel in (uarm_l_rel, farm_l_rel, hand_l_rel))
        uarm_w, farm_w, hand_w = (rel * body_w for rel in (uarm_w_rel, farm_w_rel, hand_w_rel))
        hl = uarm_l * humerus_factor
        return uarm_l, farm_l, hand_l, uarm_w, farm_w, hand_w, hl
    
    @staticmethod
    def muscle_length(q, hl, db, dt, lrte_b, lrte_t): 
        lb = sympy.sqrt((db * sympy.sin(q))**2 + (hl - db * sympy.cos(q))**2) - lrte_b
        lt = sympy.sqrt((dt * sympy.sin(q))**2 + (hl + dt * sympy.cos(q))**2) - lrte_t
        return lb, lt
    
    @staticmethod
    def moment_arm(q, lb, lt, hl, db, dt, lrte_b, lrte_t):
        rb = hl * db * sympy.sin(q) / (lb + lrte_b)
#         rb = sympy.Max(rb, 1e-2) 
        rt = - hl * dt * sympy.sin(q) / (lt + lrte_t)
#         rt = sympy.Min(rt, -1e-2) 
        return rb, rt
    
    @staticmethod
    def elbow_dynamic_properties(farm_w, farm_l, hand_w, hand_l):
        farm_com = farm_l / 2.
        hand_com = (farm_l + hand_l/2)
        resultant_mass = farm_w * farm_com + hand_w * hand_com
        inertia        = farm_w * farm_com**2 + hand_w * hand_com**2
        return resultant_mass, inertia
    
    def eval_muscle_models(self, dq, aMNb, aMNt, gMNb, gMNt, lb, lt, rb, rt, args_b, args_t, **kwargs): 
        dlb = rb * dq
        dlt = rt * dq
        
        bic = self.biceps_model.compute_output((lb, dlb), (aMNb, gMNb), args_b, **kwargs)
        tri = self.triceps_model.compute_output((lt, dlt), (aMNt, gMNt), args_t, **kwargs)
        
        return bic, tri
    
    @staticmethod
    def joint_acceleration(q, q2, Fbiceps, Ftriceps, rb, rt, farm_w, farm_l, hand_w, hand_l, g): 
        resultant_mass, inertia = ElbowModel.elbow_dynamic_properties(farm_w, farm_l, hand_w, hand_l)
        Mmuscles = - rb * Fbiceps - rt * Ftriceps
        Mgravity = - resultant_mass * g * sympy.sin(q2 - q)
        
        Mjoint   = Mmuscles + Mgravity
        ddq = Mjoint / inertia
        return ddq
    
    @NamedParamModel.unvec_args
    def compute_dynamics(self, q, dq, q2, aMNb, aMNt, gMNb, gMNt, uarm_l_rel, farm_l_rel, hand_l_rel, 
                         uarm_w_rel, farm_w_rel, hand_w_rel, body_w, body_h, humerus_factor, lrte_b, lrte_t, g, db, dt, *args): 
        args_b = args[:len(args)//2]
        args_t = args[len(args)//2:]
        
        uarm_l, farm_l, hand_l, uarm_w, farm_w, hand_w, hl =\
            ElbowModel.body_mensurations(uarm_l_rel, farm_l_rel, hand_l_rel, uarm_w_rel, farm_w_rel, 
                                         hand_w_rel, body_w, body_h, humerus_factor)
        lb, lt = ElbowModel.muscle_length(q, hl, db, dt, lrte_b, lrte_t)
        rb, rt = ElbowModel.moment_arm(q, lb, lt, hl, db, dt, lrte_b, lrte_t)
        
        Fbiceps, Ftriceps = self.eval_muscle_models(
            dq, aMNb, aMNt, gMNb, gMNt, lb, lt, rb, rt, args_b, args_t, return_rates=False)
        
        ddq    = ElbowModel.joint_acceleration(q, q2, Fbiceps, Ftriceps, rb, rt, farm_w, farm_l, hand_w, hand_l, g)
        
        y = sympy.Matrix([[dq], [ddq]])* 1e-2
        return y
    
    
    @NamedParamModel.unvec_args
    def compute_output(self,  q, dq, q2, aMNb, aMNt, gMNb, gMNt, uarm_l_rel, farm_l_rel, hand_l_rel, 
                         uarm_w_rel, farm_w_rel, hand_w_rel, body_w, body_h, humerus_factor, lrte_b, lrte_t, g, db, dt, *args): 
        args_b = args[:len(args)//2]
        args_t = args[len(args)//2:]
        
        uarm_l, farm_l, hand_l, uarm_w, farm_w, hand_w, hl =\
            ElbowModel.body_mensurations(uarm_l_rel, farm_l_rel, hand_l_rel, uarm_w_rel, farm_w_rel, 
                                         hand_w_rel, body_w, body_h, humerus_factor)
        
        lb, lt = ElbowModel.muscle_length(q, hl, db, dt, lrte_b, lrte_t)
        rb, rt = ElbowModel.moment_arm(q, lb, lt, hl, db, dt, lrte_b, lrte_t)
        
        (rateIa_b, rateIb_b), (rateIa_t, rateIb_t) = \
            self.eval_muscle_models(dq, aMNb, aMNt, gMNb, gMNt, lb, lt, rb, rt, args_b, args_t, return_force=False)
        
#         y = 
        y = sympy.Matrix([[q], [aMNb], [aMNt], [gMNb], [gMNt], [rateIa_b], [rateIb_b], [rateIa_t], [rateIb_t]])
        return y

In [117]:
elbow = ElbowModel([*params.keys()])
elbow.ix, elbow.iv, elbow.ip; 

In [35]:
pC

array([1.00000000e+00, 1.83156389e-02, 1.83156389e-02, 1.83156389e-02,
       1.00000000e+00, 1.83156389e-02, 1.83156389e-02, 1.83156389e-02,
       1.83156389e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.35462628e-04, 1.35335283e-01, 7.38905610e+00, 1.35335283e-01,
       1.35335283e-01, 1.35335283e-01, 1.35335283e-01, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.35462628e-04, 1.35335283e-01, 7.38905610e+00,
       1.35335283e-01, 1.35335283e-01, 1.35335283e-01, 1.35335283e-01,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [107]:
pE, pC, constraints = [[params[k][i] for k in param_keys] for i in range(3)] 
pE = np.asarray(pE)
pC = np.asarray(pC)

models = [
    GaussianModel(
        f=elbow.compute_dynamics, 
        g=elbow.compute_output,
        n=elbow.n, m=elbow.m, 
        pE=pE, pC=pC, V=np.array([np.exp(16)] * 5), W=np.array([np.exp(16)] * elbow.n), x=np.array([[np.pi/2.],[0]])
    ), 
    GaussianModel(l=elbow.m, v=np.zeros((elbow.m, 1)), V=np.array([np.exp(32)] * elbow.m))
]

hgm = HierarchicalGaussianModel(*models)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 1.6450881958007812e-05s)
  Compiling g... g() ok. (compiled in 1.0013580322265625e-05s)
Done. 


In [134]:
N  = 1024
dt = 1e-2
t  = np.linspace(0, N * dt, N)
aMNb = 0.5 + 0.2* np.cos(2 * np.pi * 2 * t)
aMNt = 0.5
u  = np.zeros((N, elbow.m))
u[:, 1] = aMNb

In [135]:
results = DEMInversion(hgm, states_embedding_order=4).generate(1024, u=u)

  0%|          | 0/1024 [00:00<?, ?it/s]

In [136]:
fig = make_subplots(1, 2)
color = px.colors.qualitative.Pastel + px.colors.qualitative.Safe 
names = []
for i in range(results.x.shape[2]):
    name = elbow.state_keys[i]
    fig.add_scatter(x=t, y=[*results.x[:, 0, i, 0].T], line_color=color[i], name=name, showlegend=show, row=1, col=1)
    names.append(name)

coltr = []
for i in range(results.v.shape[2]):
    name = ([*elbow.output_keys] + [*elbow.input_keys])[i]
    if name in coltr: continue
    show = name not in names
    fig.add_scatter(x=t, y=[*results.v[:, 0, i, 0].T], line_color=color[i], name=name, showlegend=show, legendgroup=name, row=1, col=2)
    names.append(name)
    coltr.append(name)
    
fig.update_layout(template='plotly_white')
fig

In [142]:
in_vE = np.array([0, 0.3, 0.1, 0, 0]).reshape((5, 1))
in_vP = np.array([np.exp(32), np.exp(4), np.exp(4), np.exp(32), np.exp(32)])

models = [
    GaussianModel(
        f=elbow.compute_dynamics, 
        g=elbow.compute_output,
        n=elbow.n, m=elbow.m, 
        pE=pE, pC=pC * 1e-32, V=np.array([np.exp(16)] * 5), W=np.array([np.exp(16)] * elbow.n), x=np.array([[np.pi/2.],[0]])
    ), 
    GaussianModel(l=elbow.m, v=in_vE, V=in_vP)
]
hgmd = HierarchicalGaussianModel(*models)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 1.3113021850585938e-05s)
  Compiling g... g() ok. (compiled in 1.0251998901367188e-05s)
Done. 


In [149]:
hgmd

[{'f': <bound method ElbowModel.compute_dynamics of <__main__.ElbowModel object at 0x7f1d36ffce50>>,
  'g': <bound method ElbowModel.compute_output of <__main__.ElbowModel object at 0x7f1d36ffce50>>,
  'fsymb': None,
  'gsymb': None,
  'm': 5,
  'n': 2,
  'l': 9,
  'p': 25,
  'x': array([[1.57079633],
         [0.        ]]),
  'v': Matrix([
  [  1.5707963267949],
  [              0.3],
  [              0.1],
  [                0],
  [                0],
  [0.602738091788146],
  [0.559689456085044],
  [0.587418087343767],
  [0.476797408386832]]),
  'pE': array([[7.00000000e+01],
         [7.62000000e-03],
         [1.37300000e-02],
         [3.61000000e-03],
         [1.75000000e+00],
         [1.64800000e-01],
         [1.95900000e-01],
         [1.14650000e-01],
         [8.00000000e-01],
         [5.23598776e-01],
         [2.82743339e+00],
         [9.81000000e+00],
         [2.00000000e-02],
         [1.20000000e-01],
         [2.39712000e+03],
         [1.50000000e-01],
         

In [147]:
y = np.zeros((N, len(elbow.output_keys)))
y[:, 0] += np.pi/2
dec = DEMInversion(hgmd).run(

E-step (F = -inf):   0%|          | 0/8 [00:00<?, ?it/s]

timestep:   0%|          | 0/1024 [00:00<?, ?it/s]

ValueError: cannot reshape array of size 1250 into shape (5,14,25)

In [148]:
import dill 
dill.settings['recurse'] = True
save_diffs = True
load_diffs = False
if save_diffs: 
    dill.dump(hgm[0].df.dx, open('.musculoskeletal_df', 'wb'))
    dill.dump(hgm[0].df.dv, open('.musculoskeletal_df', 'wb'))
    dill.dump(hgm[0].df.dp, open('.musculoskeletal_df', 'wb'))
    dill.dump(hgm[0].d2f, open('.musculoskeletal_d2f', 'wb'))    
    dill.dump(hgm[0].dg, open('.musculoskeletal_dg', 'wb'))    
    dill.dump(hgm[0].d2g, open('.musculoskeletal_d2g', 'wb'))  

if load_diffs: 
    hgm[0].df = dill.load(open('.musculoskeletal_df', 'rb'))
    hgm[0].d2f = dill.load(open('.musculoskeletal_d2f', 'rb'))
    hgm[0].dg = dill.load(open('.musculoskeletal_dg', 'rb'))
    hgm[0].d2g = dill.load(open('.musculoskeletal_d2g', 'rb'))

KeyError: 'mro'

In [ ]:
from sympy import derive_by_array

In [ ]:
X = sympy.MatrixSymbol('x', elbow.n, 1)
V = sympy.MatrixSymbol('v', elbow.m, 1)
P = sympy.MatrixSymbol('p', len(param_keys), 1)

x = np.asarray(X)
v = np.asarray(V)
p = np.asarray(P)

xvp = tuple(np.random.randn(*_.shape) for _ in (x,v,p))

f = elbow.compute_dynamics(x, v, p);
g = elbow.compute_output(x, v, p);

In [ ]:
%timeit f.jacobian(P)
%timeit g.jacobian(P)
%timeit f.jacobian(p)
%timeit g.jacobian(p)

In [ ]:
JfP = f.jacobian(P)
JgP = g.jacobian(P)
Jfp = f.jacobian(p)
Jgp = g.jacobian(p)

In [ ]:
%timeit sympy.lambdify((X, V, P), JfP, cse=True)
%timeit sympy.lambdify((X, V, P), JgP, cse=True)
%timeit sympy.lambdify((X, V, P), Jfp, cse=True)
%timeit sympy.lambdify((X, V, P), Jgp, cse=True)

In [ ]:
JfP = sympy.lambdify((X, V, P), JfP, cse=True)
JgP = sympy.lambdify((X, V, P), JgP, cse=True)
Jfp = sympy.lambdify((X, V, P), Jfp, cse=True)
Jgp = sympy.lambdify((X, V, P), Jgp, cse=True)

In [ ]:
%timeit Jfp(*xvp)
%timeit Jgp(*xvp)
%timeit JfP(*xvp)
%timeit JgP(*xvp)

In [ ]:
_ = derive_by_array(expr, p).reshape(expr.shape[0], p.shape[0])

In [ ]:
J = (expr, P)
J = J.
sympy.utilities.lambdify((X, V, P), J)(*xvp)

In [ ]:
from sympy import derive_by_array
print(sympy.lambdify((X, V, P), J)

In [ ]:
print(sympy.SparseMatrix(expr).jacobian(P))

In [ ]:
expr = elbow.compute_output(x, v, p)
# print(expr)
%timeit expr.jacobian(p)
%timeit expr.jacobian(P)

In [ ]:
expr = elbow.compute_dynamics(x, v, p)
jac = expr.jacobian(p)
sympy.lambdify((X, V, P), jac, 'numpy')(*xvp)

In [ ]:
%timeit derive_by_array(expr, p) 
jac = derive_by_array(expr, p) 
sympy.lambdify((X, V, P), jac, 'numpy')(*xvp)

In [19]:
X = sympy.symarray('x', elbow.n)
V = sympy.symarray('v', elbow.m)
P = sympy.symarray('p', len(param_keys))

x = X.reshape((-1, 1))
v = np.asarray(V).reshape((-1, 1))
p = np.asarray(P).reshape((-1, 1))

xvp = tuple(np.random.randn(*_.shape) for _ in (x,v,p))

expr = elbow.compute_output(x, v, p)
%timeit expr.jacobian(p)
jac1 = expr.jacobian(p)
print(jac1.shape)
print(sympy.lambdify((X, V, P), jac1)(*xvp))
# %timeit  derive_by_array(expr, p)
# jac2 = derive_by_array(expr, p)
# print(sympy.lambdify((X, V, P), jac2)(*xvp))

106 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(9, 45)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0]
 [0 0 0 0 array([-0.00194686]) 0 array([-0.00095353]) 0
  array([-0.00038307]) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  array([-0.00196502]) array([0.00155535]) 0 array([0.00059821]) 0 0
  array([0.00137766]) 0 0 0 0 0 0 array([0.00060911]) 0
  array([0.00038291]) array([-0.00394143]) array([0.00207414]) 0 0]
 [0 0 0 0 array([-0.53780573]) 0 array([-0.26340515]) 0
  array([-0.10582008]) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  array([0.30011688]) array([0.391

In [16]:
import ray 
ray.init()

RayContext(dashboard_url='', python_version='3.8.12', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '172.30.2.118', 'raylet_ip_address': '172.30.2.118', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-13_14-59-44_760701_286461/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-13_14-59-44_760701_286461/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-07-13_14-59-44_760701_286461', 'metrics_export_port': 47680, 'gcs_address': '172.30.2.118:43254', 'address': '172.30.2.118:43254', 'node_id': '3af4de2fbd5172ebf5674202e4373387a1b06ab3dda85116cae9acd9'})

In [39]:
np.argwhere(ret != 0)

array([[10309],
       [10311],
       [10313],
       [10330],
       [10331],
       [10333],
       [10336],
       [10343],
       [10345],
       [10346],
       [10347],
       [10399],
       [10401],
       [10403],
       [10420],
       [10421],
       [10423],
       [10426],
       [10433],
       [10435],
       [10436],
       [10437],
       [10489],
       [10491],
       [10493],
       [10510],
       [10511],
       [10513],
       [10516],
       [10523],
       [10525],
       [10526],
       [10527],
       [11254],
       [11256],
       [11258],
       [11275],
       [11276],
       [11278],
       [11281],
       [11288],
       [11290],
       [11291],
       [11292],
       [11299],
       [11301],
       [11303],
       [11320],
       [11321],
       [11323],
       [11326],
       [11333],
       [11335],
       [11336],
       [11337],
       [11389],
       [11391],
       [11393],
       [11410],
       [11411],
       [11413],
       [11416],
       [

In [ ]:
ret[13671]

In [41]:
ret = np.asarray([*starmap(lambda x, v: ray.remote(sympy.diff).remote(x,v) if x.free_symbols else x, product(jac1, P))])
fut = [*map(lambda e: isinstance(e, ray.ObjectRef), ret)]
ret[fut] = ray.get(ret[fut])
res = fut.reshape((jac1.shape[0], jac1.shape[1], P.shape[0]))

ValueError: 'object_refs' must either be an object ref or a list of object refs.

In [33]:
from itertools import starmap, product


AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
%timeit jac1.reshape(jac1.shape[0] * jac1.shape[1], 1).jacobian(p)
%timeit sympy.Matrix(np.asarray([*starmap(sympy.diff, product(jac1, p.flat))]).reshape((jac1.shape[0] * jac1.shape[1], P.shape[0])))
%timeit np.asarray([*starmap(lambda x, v: sympy.diff(x,v) if x.free_symbols else x, product(jac1, P))]).reshape((jac1.shape[0] * jac1.shape[1], P.shape[0]))

In [ ]:
mX = sympy.MatrixSymbol('X', elbow.n, 1)
mV = sympy.MatrixSymbol('V', elbow.m, 1)
mP = sympy.MatrixSymbol('P', len(param_keys), 1)
jac = jac1.xreplace(dict(zip(X, mX)))
jac = jac.xreplace(dict(zip(V, mV)))
jac = jac.xreplace(dict(zip(P, mP)))

In [ ]:
x

In [ ]:
J1 = sympy.lambdify((mX, mV, mP), jac, cse=True)
%timeit J1(*xvp)

In [42]:
import math
from tqdm.autonotebook import tqdm
from itertools import chain

import ray

def compute_sym_df_d2f(func, *dims, input_keys=None, wrt=None):
    """ 
    Use symbolic differentiation to compute jacobian and hessian of a function of 3 vectors. 
     - func: if the function to differentiate (must return a vector, ie a tensor (l, ...) where ... are empty or 1's)
     - dims: a list of tuple containing the dimensions of each argument
    Returns: (df, d2f) where: 
     - df.dx, df.dv, and df.dp contains the jacobians wrt each argument
     - d2f.dx.dx, ... contains the ndim-hessians wrt each pair of arguments

    Basically, it is faster for large expressions to derive wrt symarrays, however lambdified functions perform best
    on MatrixSymbol. Thus, we differentiate wrt symarrays, .xreplace with MatrixSymbols and then lambdify
    """
    cast = lambda x: np.array(x, dtype=np.float64)
    
    if input_keys is None: 
        import string
        input_keys = string.ascii_lowercase[:len(dims)]
    else: 
        assert(len(dims) == len(input_keys))
    
    if wrt is None:
        wrt = input_keys
    else: 
        assert(all(_ in input_keys for _ in wrt))
        
    wrt = [f'd{k}' for k in wrt]
    dims = [(dim,1) if isinstance(dim, int) else dim for dim in dims]
    
    # Squeeze column vectors
    squeezedims = [dim if len(dim) == 1 or dim[1] != 1 else (dim[0],) for dim in dims]
    
    # compute flat dimension 
    flatdims = [math.prod(dim) for dim in dims]

    # create symbolic variables
    symvars = [
        (f'd{k}', sympy.symarray(k, dim))
        for k, dim in zip(input_keys, flatdims)
    ]
    
    # create symbolic matrix symb
    symmat = [
        (f'd{k}', sympy.MatrixSymbol(k, dim, 1))
        for k, dim in zip(input_keys, flatdims)
    ]

    # symbols in column numpy arrays
    var = [
        (k, symvar.reshape((-1, 1)))
        for k, symvar in symvars
    ]    
    
    symargs = [v[1] for v in symmat]
    args = [v[1] for v in var]
    _vars = [v[1] for v in symvars]
    
    replace_dict = dict(chain(*starmap(zip, zip(_vars, symargs))))
    
    fxvp = sympy.Matrix(func(*args))
    l = fxvp.shape[0]

    
    dfsymb  = dotdict({
        d: fxvp.jacobian(sym)
        for d, sym in symvars
        if d in wrt
    })

    df  = cdotdict()
    d2f = cdotdict()


    for i, (d1, sym1) in enumerate(symvars):
        if d1 not in wrt: continue 
            
        if d1 not in d2f.keys():
            d2f[d1] = cdotdict()
            
        for j, (d2, sym2) in enumerate(symvars): 
            
            if j < i: continue
            if d2 not in wrt: continue 
                
            if d2 not in d2f.keys(): 
                d2f[d2] = cdotdict()


            ret = np.asarray([*starmap(lambda x, v: 
                    ray.remote(sympy.diff).remote(x,v) if x.free_symbols else x, 
                    product(dfsymb[d1], sym2))])
            
            future = [*map(lambda e: isinstance(e, ray.ObjectRef), ret)]
            ret[future] = ray.get(ret[future].tolist())
            
            h = ret.reshape((l, d1, d2)).tolist()
            
#             h  = (dfsymb[d1].reshape(l * sym1.shape[0], 1).jacobian(sym2))
            h  = sympy.MutableDenseNDimArray(h)
            h  = h.reshape(l, sym1.shape[0], sym2.shape[0])
            h  = sympy.Matrix(h.reshape(l, sym1.shape[0]*sym2.shape[0]))

            if len(h.free_symbols) > 0: 
                h       = h.xreplace(replace_dict)
                func_h  = sympy.lambdify(symmat, h, cse=True)

                d2f[d1][d2] = lambda *_args, _func=func_h, _target_shape=(l, *squeezedims[i], *squeezedims[j]):\
                    _func(*_args).reshape(_target_shape)

                d2f[d2][d1] = lambda *_args, _func=func_h, _interm_shape=(l, sym1.shape[0], sym2.shape[0]), _target_shape=(l, *squeezedims[j], *squeezedims[i]):\
                    _func(*_args).reshape(_interm_shape).swapaxes(1, 2).reshape(_target_shape)
            else:
                d2f[d1][d2] = lambda *_args, _symb=cast(h).reshape((l, *squeezedims[i], *squeezedims[j])): _symb
                d2f[d2][d1] = lambda *_args, _symb=cast(h).reshape((l, sym1.shape[0], sym2.shape[0])).swapaxes(1, 2).reshape((l, *squeezedims[j], *squeezedims[i])): _symb

                
        J = dfsymb[d1]
        if len(J.free_symbols) > 0:
            func_J  = sympy.lambdify(symmat, J.xreplace(replace_dict), 'numpy', cse=True)

            df[d1] = lambda *_args, _func=func_J, _target_shape=(l, *squeezedims[i]): _func(*_args).reshape(_target_shape)
        else: 
            df[d1] = lambda *_args, _symb=cast(J).reshape((l, *squeezedims[i])): _symb
        
    return df, d2f
    

In [ ]:
compute_sym_df_d2f(elbow.compute_dynamics, elbow.n, elbow.m, P.shape[0])

In [ ]:
expr = elbow.compute_dynamics(X.reshape(elbow.n, 1), V.reshape(elbow.m, 1), P.reshape(P.shape[0], 1))

In [ ]:
from sympy.printing.aesaracode import aesara_function
f = aesara_function([X, V, P], [expr])

In [ ]:
[*map(lambda _: sympy.diff(_, p), expr.reshape(2, 1))]

In [ ]:
jac1[1 , 3], p))

In [ ]:
[*product(range(3), range(3))]

In [ ]:
from itertools import product, starmap, combinations_with_replacement
jac = [*starmap(lambda x,y: sympy.diff(x, y) if x.free_symbols else x, product(expr, P))]
jac = np.asarray(jac).reshape((-1, len(P)))

In [ ]:
from joblib import Parallel, delayed
from functools import lru_cache
from itertools import repeat, chain

@lru_cache
def jac_it(f, v): 
    return starmap(lambda x,y: sympy.diff(x, y) if x.free_symbols else x, product(f, v))

def jacobian(f, v): 
    return [*jac_it(f, v)] 
    
def df_d2f(f, *args, njobs=1): 
    j = [*starmap(jacobian, zip(repeat(f), args))]
    
    if njobs > 1: 
        h = [Parallel(njobs)(starmap(delayed(jacobian), zip(repeat(_), args))) for _ in j]
    else:
        
        h = [[*starmap(jacobian, zip(repeat(_), args))] for _ in j]
    return j, h

In [ ]:
%timeit jacobian(jacobian(expr,P), X)

In [ ]:
d2fdpdp = jacobian(jacobian(expr,P), P)

In [ ]:
sympy.lambdify((X, V, P), d2fdpdp, cse=True)(*xvp)

In [ ]:
%timeit jacobian(jacobian(expr,P), P)

In [ ]:
j, h = df_d2f(J, X, V, P, njobs=1)

In [ ]:
h

In [ ]:
j, df_d2f(J, P, (X, V, P), njobs=11)

In [ ]:
J = [*jacobian_it(expr, X)]
# %timeit df_d2f(J, P)
%timeit df_d2f(J, P, (X, V, P), njobs=11)


In [ ]:
he

In [ ]:
def hessian(dfdv, v): 
    hes = [
        *map(
            lambda f: 
            [*starmap(
                lambda i, j, _P=P:sympy.diff(dfdv[i], v[j]), 
                combinations_with_replacement(range(len(v)), 2))],
            jac)
    ]
    return hes

In [ ]:
%timeit [*map(lambda f: [*starmap(lambda i, j, _P=P:sympy.diff(f[i], P[j]), combinations_with_replacement(range(len(P)), 2))], jac)]

In [ ]:
[*map(lambda f: [*starmap(lambda i, j, _P=P:sympy.diff(f[i], P[j]), combinations_with_replacement(range(len(P)), 2))], jac)]

In [ ]:
print([*map(lambda _: _.jacobian(p), jac1)])

In [ ]:
jac1.reshape(2 * 45, 1).jacobian(p)

In [ ]:
compute_sym_df_d2f(elbow.compute_dynamics, elbow.n, elbow.m, len(param_keys))

In [ ]:
from math import prod
def compute_sym_df_d2f(func, *dims, input_keys=None, wrt=None, cast_to=np.ndarray, wrap_type='lambdify'):
    """ 
    Use symbolic differentiation to compute jacobian and hessian of a function of 3 vectors. 
     - func: if the function to differentiate (must return a vector, ie a tensor (l, ...) where ... are empty or 1's)
     - dims: a list of tuple containing the dimensions of each argument
    Returns: (df, d2f) where: 
     - df.dx, df.dv, and df.dp contains the jacobians wrt each argument
     - d2f.dx.dx, ... contains the ndim-hessians wrt each pair of arguments
    """
    if cast_to == np.ndarray: 
        cast = lambda x: np.array(x, dtype=np.float64)
    elif callable(cast_to):
        cast = cast_to
    else: raise NotImplementedError()
        
    if input_keys is None: 
        import string
        input_keys = string.ascii_lowercase[:len(dims)]
    else: 
        assert(len(dims) == len(input_keys))
    
    if wrt is None:
        wrt = input_keys
    else: 
        assert(all(_ in input_keys for _ in wrt))
        
    wrt = [f'd{k}' for k in wrt]
    dims = [(dim,1) if isinstance(dim, int) else dim for dim in dims]
    
    # Squeeze column vectors
    squeezedims = [dim if len(dim) == 1 or dim[1] != 1 else (dim[0],) for dim in dims]
    
    # compute flat dimension 
    flatdims = [prod(dim) for dim in dims]

    # create symbolic variables
    symvars = [
        (f'd{k}', sympy.symarray(k, dim))
        for k, dim in zip(input_keys, flatdims)
    ]

    # wrap the symbols in numpy arrays
    var = [
        (k, np.array(symvar).reshape(dim))
        for (k, symvar), dim in zip(symvars, dims)
    ]
    
    # create flat numpy variables (for differentiation)
    # flatvar = [
    #     (k, v.reshape((flatdim,)))
    #     for (k, v), flatdim in zip(var, flatdims)
    # ]
    
    
    symargs = [v[1] for v in symvars]
    args = [v[1] for v in var]


    fxvp = sympy.Matrix(func(*args))
    l = fxvp.shape[0]

    dfsymb  = dotdict({
        d: fxvp.jacobian(sym)
        for d, sym in var
        if d in wrt
    })
    print('jac ok.')

    df  = cdotdict()
    d2f = cdotdict()

    for i, (d1, sym1) in enumerate(var):
        if d1 not in wrt: continue 
            
        if d1 not in d2f.keys():
            d2f[d1] = cdotdict()
            
        for j, (d2, sym2) in enumerate(var): 
            if j < i: continue
            if d2 not in wrt: continue 
                
            if d2 not in d2f.keys(): 
                d2f[d2] = cdotdict()

            h  = sympy.MutableDenseNDimArray((dfsymb[d1].reshape(l * sym1.shape[0], 1).jacobian(sym2)))
            h  = h.reshape(l, sym1.shape[0], sym2.shape[0])
            ht = sympy.permutedims(h, (0, 2, 1))
            
            h  = sympy.Matrix(h.reshape(l, sym1.shape[0]*sym2.shape[0]))
            ht = sympy.Matrix(h.reshape(l, sym1.shape[0]*sym2.shape[0]))
            
            if len(h.free_symbols) > 0: 
                if wrap_type == 'autowrap':
                    func_h  = autowrap(h, args=symargs)
                    func_ht = autowrap(ht, args=symargs)
                elif wrap_type == 'lambdify': 
                    func_h  = sympy.lambdify(symargs, h, 'numpy')
                    func_ht = sympy.lambdify(symargs, ht, 'numpy')

                d2f[d1][d2] = lambda *_args, _func=func_h, _target_shape=(l, *squeezedims[i], *squeezedims[j]):\
                    _func(*_args).reshape(_target_shape)
                d2f[d2][d1] = lambda *_args, _func=func_ht, _target_shape=(l, *squeezedims[j], *squeezedims[i]):\
                    _func(*_args).reshape(_target_shape)
            else:
                d2f[d1][d2] = lambda *_args, _symb=cast(h).reshape((l, *squeezedims[i], *squeezedims[j])): _symb
                d2f[d2][d1] = lambda *_args, _symb=cast(ht).reshape((l, *squeezedims[j], *squeezedims[i])): _symb
                
        J = dfsymb[d1]
        if len(J.free_symbols) > 0:
            if wrap_type == 'autowrap':
                func_J  = autowrap(J, args=symargs)
            elif wrap_type == 'lambdify': 
                func_J  = sympy.lambdify(symargs, J, 'numpy')

            df[d1] = lambda *_args, _func=func_J, _target_shape=(l, *squeezedims[i]): _func(*_args).reshape(_target_shape)
        else: 
            df[d1] = lambda *_args, _symb=cast(J).reshape((l, *squeezedims[i])): _symb
        
    return df, d2f
    


In [ ]:
expr = elbow.compute_dynamics(x, v, p)

In [ ]:
sympy.reshape(sympy.symarray('x', elbow.n), (elbow.n,1))

In [ ]:
expr = elbow.compute_output(x, v, p)


In [ ]:
J0 = expr.jacobian(P)

In [ ]:
J0 = J0.as_immutable()

In [ ]:
J0.as_explicit()

In [ ]:
compute_sym_df_d2f(func

In [ ]:
from sympy.tensor.array import derive_by_array

J = derive_by_array(expr, X)

In [ ]:
%timeit H = derive_by_array(derive_by_array(expr, p), p)

In [ ]:
%timeit sympy.lambdify((X, V, P), H)

In [ ]:
%timeit sympy.Matrix(expr.jacobian(P).as_immutable().as_explicit()).jacobian(P)

In [ ]:
%timeit sympy.lambdify((X, V, P), expr.jacobian(P))
%timeit sympy.lambdify((X, V, P), sympy.cse(expr.jacobian(P), list=False)[1])
%timeit sympy.lambdify((X, V, P), sympy.cse(expr, list=False)[1].jacobian(p))

In [ ]:
_x = np.random.randn(*x.shape)
_v = np.random.randn(*v.shape)
_p = np.random.randn(*p.shape)

In [ ]:
J0 =  sympy.lambdify((X, V, P), expr.jacobian(P))
J1 = sympy.lambdify((X, V, P), sympy.cse(expr.jacobian(P), list=False)[1])
J2 = sympy.lambdify((X, V, P), sympy.cse(expr, list=False)[1].jacobian(p))

In [ ]:
print(J0(_x, _v, _p))
# print(J1(_x, _v, _p))
print(J2(_x, _v, _p))

In [ ]:
import dill
dill.settings['recurse'] = True
save_deriv=True
load_deriv=False
if load_deriv: 
    for _ in ('df', 'd2f', 'dg', 'd2g'): 
        models[0][_] = dill.load(open(f'.musculoskeletal_{_}_ser', 'rb'))
if save_deriv: 
    for _ in ('df', 'd2f', 'dg', 'd2g'):
        dill.dump(models[0][_], open(f'.musculoskeletal_{_k}_ser', 'rb'))

In [ ]:
results = DEMInversion(hgm, states_embedding_order=2).generate(100)

In [ ]:
px.line(y=[*results.v[:, 0, :, 0].T])

In [ ]:
J00 = sympy.lambdify([X, V, P], j0[1], 'numpy', cse=True)
J10 = sympy.lambdify([X, V, P], j1[0], 'numpy', cse=True)

In [ ]:
def jacobians(expr, *args): 
    subs, simplexpr = sympy.cse(expr, list=False, optimizations='basic')
    symb, transf = ([s[i] for s in subs] for i in range(2))
    
    mtransf = [sympy.Matrix([[tr]]) for tr in transf]
    nvar   = sympy.Matrix(symb)

    dfds = simplexpr.jacobian(nvar)

    J = []
    for var in args:
        sprev = []
        Jtr_vecs = []
        for s, tr in zip(symb, mtransf): 
            dsdx = tr.jacobian(var)
        
            if len(sprev) > 0: 
                _sp = sympy.SparseMatrix(np.stack([sprev]))
                dsdsp = tr.jacobian(_sp)
                dspdx = sympy.SparseMatrix(np.concatenate(Jtr_vecs))
                dsdx += dsdsp @ dspdx
#                 print(dsdx.jacobian(var))
                
            sprev.append(s)
            Jtr_vecs.append(dsdx)

        dsdx = sympy.SparseMatrix(np.concatenate(Jtr_vecs))
        dfdx = dfds @ dsdx
        for s, tr in zip(symb, transf): 
            dfds.subs(s, tr)
        J.append(dfdx)
        
    return J

j1 = jacobians(elbow.compute_dynamics(x, v, p), X, V, P)
print(j1[0])
# for new, expr in subs:
#     var = 

In [ ]:
print(j0[0][0, 0])

In [ ]:
sympy.Matrix(np.zeros((3, 3))).as_e

In [ ]:
gen = DEMInversion(hgm).generate(50)